# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [872]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [819]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [797]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('ethnicgroup').text)

Albanian
Greek
Macedonian
Serb
Montenegrin
Albanian
Spanish


In [798]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [884]:
document = ET.parse( './data/mondial_database.xml' )

In [885]:
import pandas as pd

# Create empty lists for both country and infant_mortality
country = []
infant_mortality = []

# print names of all countries and their infant mortality rates
for element in document.iterfind('country'):
    #print('* ' + element.find('name').text + ':'),
    country.append(element.find('name').text)
    capitals_string = ''
    try:
        for subelement in element.getiterator('country'):
            infant_mortality.append(subelement.find('infant_mortality').text)
    except:
        infant_mortality.append('')

# Build a Pandas data from with country and infant mortality columns
df = pd.DataFrame({'Country': country,'Infant Mortality':infant_mortality})      

# Remove the rows in which there is no infant mortality record
df = df[df['Infant Mortality'] != '']

# Change the data type form string to float so that the data frame can be sorted appropriately
df['Infant Mortality'] = df['Infant Mortality'].astype(float, errors = 'coerce')

# Print the 10 lowest countries infant mortality rate
df.sort_values('Infant Mortality').head(10)


#len(country_list)

,Country,Infant Mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [886]:
# instantiate city and population lists
city_list = []
population_list = []

# iteratively find the country name
for element in document.iterfind('country'):
    #print('* ' + element.find('name').text + ':'),

        for subelement in element.getiterator('city'):
            city_list.append(subelement.find('name').text) 
            try:
                # extract the latest population record
                population_list.append(subelement.find('population[last()]').text) 
            except:
                # if there is no population record add an empty string to the list
                population_list.append('')


# Create a City and Population dataframe
df = pd.DataFrame({'City': city_list,'Population':population_list}) 

# Remove cities that do not have a population record
df = df[df['Population'] != '']

# coerce the population data type from string to float
df['Population'] = df['Population'].astype(float, errors = 'coerce')

# Print the 10 highest populated cities
df.sort_values('Population').tail(10)

#len(city_list)

,City,Population
1067,Shenzhen,10358381.0
1582,Delhi,11034555.0
1064,Guangzhou,11071424.0
1342,Tianjin,11090314.0
2810,São Paulo,11152344.0
1340,Beijing,11716620.0
479,Moskva,11979529.0
1527,Mumbai,12442373.0
771,Istanbul,13710512.0
1341,Shanghai,22315474.0


In [887]:
# Create empty country population lists
country_list = []
CNTRYpopulation_list = []

# Extract country population data from mondial database
for element in document.iterfind('country'):
    country_list.append(element.find('name').text)
    
    try:
        CNTRYpopulation_list.append(element.find('population[last()]').text) 
    except:
        CNTRYpopulation_list.append('')

# Create dataframe from country population lists, cast as float and sort
df = pd.DataFrame({'City': country_list,'Population':CNTRYpopulation_list}) 
df = df[df['Population'] != '']
df['Population'] = df['Population'].astype(float, errors = 'coerce')        

In [888]:
## Question 3, find the highest populations of ethnic groups 

# Extract ethnic group and population data from mondial database
for i in range(1,13):
    for element in document.iterfind('country'):
        try:
            EthnGrp.append(element.find('ethnicgroup[i]').text)
            EthPct.append(element.find('ethnicgroup[i]').get('percentage'))
            country_population.append(element.find('population[last()]').text)
        except:
            EthnGrp.append('')
            EthPct.append('')
            country_population.append('')

# Create city population dataframe
df = pd.DataFrame({'CountryPop': country_population,
                  'EthGrp': EthnGrp, 'EthPct': EthPct}) 

# Remove empty Ethnic Groups, cast the country pop from string to numeric and derive the count of ethic group from country
df = df[df['EthGrp'] != ''] 
df['CountryPop'] = pd.to_numeric(df['CountryPop'])
df['EthCnt'] = pd.to_numeric(df['EthPct'])/100 * df['CountryPop']

# Sort and sum ethnic population by ethnic group
t = df.groupby('EthGrp').sum().sort_values('EthCnt').tail(10)
pd.DataFrame(t['EthCnt'].iloc[::-1])

,EthCnt
EthGrp,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [941]:
# Create empty river lists
river_list = []
river_length = []
country_code = []

# Extract river data from mondial database
for element in document.iterfind('river'):
    try:
        river_list.append(element.find('name').text)
    except:
        river_list.append('')        
    try:
        river_length.append(element.find('length').text)
#            country_code.append(element.get('country'))
    except:
        river_length.append('')
#    country_code.append('')
    try:
        country_code.append(element.get('country'))
    except:
        country_code.append('') 

# Create river data frame
df = pd.DataFrame({'Country_Name': country_code,'River_Name': river_list, 'River_Length': river_length}) # 

# Remove missing river names and cast river length from string to numeric
df = df[df['River_Name'] != ''] 
df['River_Length'] = pd.to_numeric(df['River_Length'])

# drop na values
df = df.dropna()

# sort and get the last 10 values
t = df.sort_values('River_Length').tail(10)
a = pd.DataFrame(t.iloc[::-1])

# Replace the country code with the country name
a.set_value(174, 'Country_Name', 'Colombia Brazil Peru')
a.set_value(137, 'Country_Name', 'China')
a.set_value(136, 'Country_Name', 'China')
a.set_value(123, 'Country_Name', 'Russia')
a.set_value(205, 'Country_Name', 'Congo Zaire')
a.set_value(138, 'Country_Name', 'China Laos Thailand Cambodia Vietnam')
a.set_value(115, 'Country_Name', 'Russia Kazakhstan China')
a.set_value(186, 'Country_Name', 'Mali Niger Nigeria Guinea')
a.set_value(160, 'Country_Name', 'United States')
a.set_value(119, 'Country_Name', 'Russia')

,Country_Name,River_Length,River_Name
174,Colombia Brazil Peru,6448.0,Amazonas
137,China,6380.0,Jangtse
136,China,4845.0,Hwangho
123,Russia,4400.0,Lena
205,Congo Zaire,4374.0,Zaire
138,China Laos Thailand Cambodia Vietnam,4350.0,Mekong
115,Russia Kazakhstan China,4248.0,Irtysch
186,Mali Niger Nigeria Guinea,4184.0,Niger
160,United States,4130.0,Missouri
119,Russia,4092.0,Jenissej


In [962]:
# Create empty lake lists
lake_list = []
lake_area = []
country_code = []

# Extract lake data from the mondial database
for element in document.iterfind('lake'):
    try:
        lake_list.append(element.find('name').text)
    except:
        lake_list.append('')        
    try:
        lake_area.append(element.find('area').text)
    except:
        lake_area.append('')
#    country_code.append('')
    try:
        country_code.append(element.get('country'))
    except:
        country_code.append('') 

# Create a pandas data frame from the lake lists
df = pd.DataFrame({'Country_Name': country_code,'Lake_Name': lake_list, 'Lake_Area': lake_area}) # 

# cast Lake_Area from String to Numeric
df['Lake_Area'] = pd.to_numeric(df['Lake_Area'])

# Drop na values
df = df.dropna()

# reverse order, last 10 records
t = df.sort_values('Lake_Area').tail(10)
t = pd.DataFrame(t.iloc[::-1])

# Replace Country Codes with Country Name
t.set_value(54, 'Country_Name', 'Russia Azerbaijan Kazakhstan Iran Turkmenistan')
t.set_value(109, 'Country_Name', 'Canada United States')
t.set_value(81, 'Country_Name', 'Tanzania Kenya Uganda')
t.set_value(106, 'Country_Name', 'Canada United States')
t.set_value(108, 'Country_Name', 'United States')
t.set_value(47, 'Country_Name', 'Israel Jordan West Bank')
t.set_value(83, 'Country_Name', 'Zaire Zambia Burundi Tanzania')
t.set_value(98, 'Country_Name', 'Canada')
t.set_value(43, 'Country_Name', 'Russia')
t.set_value(89, 'Country_Name', 'Malawi Tanzania Mozambique')

,Country_Name,Lake_Area,Lake_Name
54,Russia Azerbaijan Kazakhstan Iran Turkmenistan,386400.0,Caspian Sea
109,Canada United States,82103.0,Lake Superior
81,Tanzania Kenya Uganda,68870.0,Lake Victoria
106,Canada United States,59600.0,Lake Huron
108,United States,57800.0,Lake Michigan
47,Israel Jordan West Bank,41650.0,Dead Sea
83,Zaire Zambia Burundi Tanzania,32893.0,Lake Tanganjika
98,Canada,31792.0,Great Bear Lake
43,Russia,31492.0,Ozero Baikal
89,Malawi Tanzania Mozambique,29600.0,Lake Malawi


In [960]:
airport_list = []
airport_elev = []
country_code = []

for element in document.iterfind('airport'):
    try:
        airport_list.append(element.find('name').text)
    except:
        airport_list.append('')        
    try:
        airport_elev.append(element.find('elevation').text)
#            country_code.append(element.get('country'))
    except:
        airport_elev.append('')
#    country_code.append('')
    try:
        country_code.append(element.get('country'))
    except:
        country_code.append('') 


df = pd.DataFrame({'Country_Name': country_code,'Airport_Name': airport_list, 'Airport_Elevation': airport_elev}) # 

df = df[df['Airport_Name'] != ''] 
df['Airport_Elevation'] = pd.to_numeric(df['Airport_Elevation'])

df
df = df.dropna()
t = df.sort_values('Airport_Elevation').tail(10)
t = pd.DataFrame(t.iloc[::-1])

t.set_value(80, 'Country_Name', 'Bolivia')
t.set_value(219, 'Country_Name', 'China')
t.set_value(241, 'Country_Name', 'China')
t.set_value(813, 'Country_Name', 'Peru')
t.set_value(815, 'Country_Name', 'Peru')
t.set_value(82, 'Country_Name', 'Bolivia')
t.set_value(334, 'Country_Name', 'Ecuador')
t.set_value(805, 'Country_Name', 'Peru')
t.set_value(807, 'Country_Name', 'Peru')
t.set_value(692, 'Country_Name', 'Mexico')



,Airport_Elevation,Airport_Name,Country_Name
80,4063.0,El Alto Intl,Bolivia
219,4005.0,Lhasa-Gonggar,China
241,3963.0,Yushu Batang,China
813,3827.0,Juliaca,Peru
815,3311.0,Teniente Alejandro Velasco Astete Intl,Peru
82,2905.0,Juana Azurduy De Padilla,Bolivia
334,2813.0,Mariscal Sucre Intl,Ecuador
805,2719.0,Coronel Fap Alfredo Mendivil Duarte,Peru
807,2677.0,Mayor General FAP Armando Revoredo Iglesias Ai...,Peru
692,2581.0,Licenciado Adolfo Lopez Mateos Intl,Mexico
